<a href="https://colab.research.google.com/github/dsteele101/colab_notebooks/blob/main/OpenAI_Weather_Function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Test of OpenAI functions.

Sign up for a free account here for the Weather API https://www.weatherapi.com/.

In [ ]:
!pip install openai
!pip install requests

In [5]:
import openai
import json
import requests

In [6]:
WEATHER_API_KEY = ""
openai.api_key = ""

In [7]:
def get_current_weather(location, unit="fahrenheit"):
    url = f"http://api.weatherapi.com/v1/current.json?key={WEATHER_API_KEY}&q={location}&aqi=no"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        temp_fahrenheit = data["current"]["temp_f"]
        condition_text = data["current"]["condition"]["text"]
        unit = "Fahrenheit"

        weather_info = {
            "location": location,
            "temperature": str(temp_fahrenheit),
            "unit": unit,
            "forecast": [condition_text],
        }

        json_data = json.dumps(weather_info)
        return json_data
    else:
        print("Error:", response.status_code)

In [8]:
def run_conversation():
    messages = [{"role": "user", "content": "What's the weather like in Austin?"}]
    functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        }
    ]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        function_call="auto",
    )
    response_message = response["choices"][0]["message"]

    if response_message.get("function_call"):
        available_functions = {
            "get_current_weather": get_current_weather,
        }
        function_name = response_message["function_call"]["name"]
        fuction_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        function_response = fuction_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit"),
        )

        messages.append(response_message)
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
        )
        return second_response

In [9]:
print(run_conversation())

{
  "id": "chatcmpl-7VpxTbdygZKqxObjIfYcXd0Peym0D",
  "object": "chat.completion",
  "created": 1687822251,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The weather in Austin is currently sunny with a temperature of 100.9 degrees Fahrenheit."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 72,
    "completion_tokens": 18,
    "total_tokens": 90
  }
}
